#We extract encodings from sentiment, humor, irony and political stance BERT Models for our multimodal network

In [ ]:
!pip install transformers
!pip install sentence_transformers
!pip install sentencepiece --upgrade
!pip install emoji

In [ ]:
import pandas as pd
df_train = pd.read_csv('/content/drive/MyDrive/meme_classification_data/train.csv')
df_train.head(3)

,ID,image id,text,label,label_num
0,1,image_2455.jpg,- It is not our fight - Are we not part of thi...,troll,2
1,2,image_3701.jpg,THAT'S THE DIFFERENCE BETWEEN YOU AND ME YOU...,none,0
2,3,image_4166.png,- WHAT DO THE TITANIC AND THE SIXTH SENSE HAVE...,none,0


#Extracting humor features (256/sample)

In [ ]:
import keras
from keras.models import Model
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop,Adam
model_tmp = keras.models.load_model("/content/drive/MyDrive/models/colbert-trained/")
model= Model(inputs=model_tmp.input, outputs=model_tmp.layers[-2].output)
model.trainable = False
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_19 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_20 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_21 (InputLayer)           [(None, 20)]         0                                            
__________________________________________________________________________________________________
input_22 (InputLayer)           [(None, 20)]         0                                            
______________________________________________________________________________________________

In [ ]:
import subprocess
from ast import literal_eval

def run(command):
    process = subprocess.Popen(command, shell=True, stdout=subprocess.PIPE)
    out, err = process.communicate()
    print(out.decode('utf-8').strip())

print('# CPU')
run('cat /proc/cpuinfo | egrep -m 1 "^model name"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu MHz"')
run('cat /proc/cpuinfo | egrep -m 1 "^cpu cores"')

print('# RAM')
run('cat /proc/meminfo | egrep "^MemTotal"')

print('# GPU')
run('lspci | grep VGA')

print('# OS')
run('uname -a')

# CPU
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
cpu MHz		: 2299.998
cpu cores	: 1
# RAM
MemTotal:       13333564 kB
# GPU

# OS
Linux 290561af34d3 4.19.112+ #1 SMP Thu Jul 23 08:00:38 PDT 2020 x86_64 x86_64 x86_64 GNU/Linux


In [ ]:
import pandas as pd
import numpy as np
import sklearn
from sklearn.model_selection import GroupKFold
import matplotlib.pyplot as plt
from tqdm.notebook import tqdm
import tensorflow as tf
import tensorflow.keras.backend as K
from tensorflow import keras 

import os
from scipy.stats import spearmanr
from math import floor, ceil
from transformers import *

import seaborn as sns
import string
import re    #for regex

np.set_printoptions(suppress=True)
print(tf.__version__)
training_sample_count = 1991 
test_count = 600

MAX_SENTENCE_LENGTH = 20
MAX_SENTENCES = 5
MAX_LENGTH = 100

2.4.1


In [ ]:
def hu(label_no):
  if label_no == 0:
    return 0
  else:
     return 1

df_train['Humor'] = df_train['label_num'].apply(hu)

In [ ]:
output_categories = ['Humor']
input_categories = ['text']

from transformers import BertTokenizer

MODEL_TYPE = 'bert-base-uncased'
tokenizer = BertTokenizer.from_pretrained(MODEL_TYPE)

import nltk
nltk.download('punkt')
from nltk.tokenize import sent_tokenize

def return_id(str1, str2, truncation_strategy, length):

    inputs = tokenizer.encode_plus(str1, str2,
        add_special_tokens=True,
        max_length=length,
        truncation_strategy=truncation_strategy)

    input_ids =  inputs["input_ids"]
    input_masks = [1] * len(input_ids)
    input_segments = inputs["token_type_ids"]
    padding_length = length - len(input_ids)
    padding_id = tokenizer.pad_token_id
    input_ids = input_ids + ([padding_id] * padding_length)
    input_masks = input_masks + ([0] * padding_length)
    input_segments = input_segments + ([0] * padding_length)

    return [input_ids, input_masks, input_segments]


def compute_input_arrays(df, columns, tokenizer):
    model_input = []
    for xx in range((MAX_SENTENCES*3)+3):
        model_input.append([])
    
    for _, row in tqdm(df[columns].iterrows()):
        i = 0
        
        # sent
        sentences = sent_tokenize(row.text)
        for xx in range(MAX_SENTENCES):
            s = sentences[xx] if xx<len(sentences) else ''
            ids_q, masks_q, segments_q = return_id(s, None, 'longest_first', MAX_SENTENCE_LENGTH)
            model_input[i].append(ids_q)
            i+=1
            model_input[i].append(masks_q)
            i+=1
            model_input[i].append(segments_q)
            i+=1
        
        # full row
        ids_q, masks_q, segments_q = return_id(row.text, None, 'longest_first', MAX_LENGTH)
        model_input[i].append(ids_q)
        i+=1
        model_input[i].append(masks_q)
        i+=1
        model_input[i].append(segments_q)
        
    for xx in range((MAX_SENTENCES*3)+3):
        model_input[xx] = np.asarray(model_input[xx], dtype=np.int32)
        
    print(model_input[0].shape)
    return model_input


[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [ ]:
train_inputs = compute_input_arrays(df_train, input_categories, tokenizer)


Truncation was not explicitly activated but `max_length` is provided a specific value, please use `truncation=True` to explicitly truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.



(1991, 20)


In [ ]:
humor_feats_train = model.predict(train_inputs)
cols = ["Humor"+str(i) for i in range(256)]
humor_train = pd.DataFrame(humor_feats_train, columns = cols)
humor_train

,Humor0,Humor1,Humor2,Humor3,Humor4,Humor5,Humor6,Humor7,Humor8,Humor9,Humor10,Humor11,Humor12,Humor13,Humor14,Humor15,Humor16,Humor17,Humor18,Humor19,Humor20,Humor21,Humor22,Humor23,Humor24,Humor25,Humor26,Humor27,Humor28,Humor29,Humor30,Humor31,Humor32,Humor33,Humor34,Humor35,Humor36,Humor37,Humor38,Humor39,...,Humor216,Humor217,Humor218,Humor219,Humor220,Humor221,Humor222,Humor223,Humor224,Humor225,Humor226,Humor227,Humor228,Humor229,Humor230,Humor231,Humor232,Humor233,Humor234,Humor235,Humor236,Humor237,Humor238,Humor239,Humor240,Humor241,Humor242,Humor243,Humor244,Humor245,Humor246,Humor247,Humor248,Humor249,Humor250,Humor251,Humor252,Humor253,Humor254,Humor255
0,0.233748,1.124836,0.000000,0.0,0.000000,0.967323,0.0,0.000000,0.000000,0.750813,0.000000,0.000000,0.000000,0.399244,0.000000,0.389811,0.000000,0.000000,0.000000,0.961440,0.000000,0.0,0.824324,0.359689,0.469073,0.000000,1.721427,0.000000,0.000000,0.358870,0.304767,0.000000,0.075147,0.945581,0.000000,0.000000,0.000000,3.088200,0.126276,0.0,...,1.223343,0.000000,0.343005,0.946816,0.0,0.643143,0.662584,2.239466,0.0,0.000000,0.729140,0.677913,1.175695,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.095678,0.890213,0.112187,1.055337,1.077465,0.687913,0.806087,0.828289,1.021699,0.506017,0.000000,2.040450,1.845617,0.0,0.0,0.274200,0.000000,0.0,0.0,0.0,0.0
1,0.000000,0.935769,0.000000,0.0,0.000000,1.105169,0.0,0.000000,0.000000,0.959850,0.000000,0.000000,0.000000,0.683894,0.000000,1.450330,0.274527,0.000000,0.000000,1.591888,0.000000,0.0,0.954200,0.367212,0.000000,0.000000,1.801471,0.337876,0.000000,0.203592,0.130740,0.000000,0.044777,1.661291,0.000000,0.000000,0.641644,3.400157,0.665272,0.0,...,2.298942,0.000000,1.248765,0.926081,0.0,1.424482,1.748128,3.144329,0.0,0.000000,0.800955,0.214437,1.723894,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,1.282011,0.124401,0.054556,1.077026,1.332671,0.632204,0.922491,0.648001,1.306715,0.556412,0.000000,2.157296,1.948588,0.0,0.0,0.869371,0.000000,0.0,0.0,0.0,0.0
2,0.000000,1.182600,0.000000,0.0,0.000000,1.294058,0.0,0.000000,0.000000,1.123085,0.000000,0.000000,0.000000,0.623180,0.000000,1.452006,0.436205,0.000000,0.000000,1.840212,0.000000,0.0,1.003166,0.104828,0.000000,0.000000,1.866113,0.406008,0.000000,0.285807,0.246883,0.000000,0.165655,1.530939,0.000000,0.000000,0.623480,3.488409,0.705259,0.0,...,2.297391,0.000000,1.197551,1.307207,0.0,1.341542,1.582172,3.391779,0.0,0.000000,0.789351,0.286955,1.629585,0.000000,0.000000,0.000000,0.0,0.000000,0.003319,1.333921,0.000000,0.137039,1.458906,1.495157,0.850295,0.845505,0.464582,1.669338,0.338863,0.000000,2.292515,2.063491,0.0,0.0,0.698878,0.000000,0.0,0.0,0.0,0.0
3,0.000000,0.998556,0.000000,0.0,0.000000,1.164469,0.0,0.000000,0.000000,1.126349,0.000000,0.000000,0.000000,0.617838,0.000000,1.324066,0.327008,0.000000,0.000000,1.550293,0.000000,0.0,1.114334,0.200161,0.095917,0.000000,1.809035,0.222456,0.000000,0.324349,0.260884,0.000000,0.143522,1.390524,0.000000,0.000000,0.569097,3.518647,0.606445,0.0,...,2.081265,0.000000,1.033181,1.082559,0.0,1.216077,1.512574,3.119587,0.0,0.000000,0.921958,0.253740,1.488370,0.000000,0.000000,0.000000,0.0,0.000000,0.087249,1.236882,0.184693,0.101759,1.455147,1.484358,0.630011,0.750303,0.489510,1.551056,0.492316,0.000000,2.289779,2.039574,0.0,0.0,0.571754,0.000000,0.0,0.0,0.0,0.0
4,0.000000,0.777744,0.000000,0.0,0.000000,0.833131,0.0,0.000000,0.000000,0.780882,0.000000,0.000000,0.000000,0.517889,0.000000,0.850987,0.185724,0.000000,0.000000,1.313825,0.000000,0.0,0.953452,0.431325,0.078734,0.000000,1.610922,0.066797,0.000000,0.236553,0.106516,0.000000,0.003822,1.368062,0.000000,0.000000,0.456471,3.142956,0.477006,0.0,...,1.694385,0.000000,0.729752,0.796741,0.0,1.199223,1.302080,2.543799,0.0,0.000000,0.762166,0.021772,1.301226,0.000000,0.000000,0.000000,0.0,0.000000,0.097807,0.984522,0.281788,0.015301,0.735671,1.204009,0.618267,0.676121,0.571191,1.265971,0.453850,0.000000,2.005636,1.868932,0.0,0.0,0.557955,0.000000,0.0,0.0,0.0,0.0
...,...,...,...,...,

In [ ]:
humor_train.to_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Humor.csv', index=False)


#Sentiment Features (768/sample)

In [ ]:
import nltk
import numpy as np
from sentence_transformers import SentenceTransformer
st = SentenceTransformer('distilbert-base-uncased-finetuned-sst-2-english') 
sentiment_feats_train = []
for index, row in df_train.iterrows():
  sentiment_feats_train.append(st.encode(row['text']))
sentiment_feats_train = np.array(sentiment_feats_train)
sentiment_feats_train.shape

In [ ]:
cols = ['Sentiment'+str(i) for i in range(768)]
sentiment_train = pd.DataFrame(sentiment_feats_train, columns = cols)
sentiment_train

In [ ]:
sentiment_train.to_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Sentiment.csv', index=False)

#Irony Featrues (768/sample)

In [ ]:
import nltk
import numpy as np
from sentence_transformers import SentenceTransformer
st = SentenceTransformer('cardiffnlp/twitter-roberta-base-irony')
irony_feats_train = []
for index, row in df_train.iterrows():
  irony_feats_train.append(st.encode(row['text']))
irony_feats_train = np.array(irony_feats_train)
irony_feats_train.shape

Exception when trying to download http://sbert.net/models/cardiffnlp/twitter-roberta-base-irony.zip. Response 404
SentenceTransformer-Model http://sbert.net/models/cardiffnlp/twitter-roberta-base-irony.zip not found. Try to create it from scratch
Try to create Transformer Model cardiffnlp/twitter-roberta-base-irony with mean pooling


Some weights of RobertaModel were not initialized from the model checkpoint at cardiffnlp/twitter-roberta-base-irony and are newly initialized: ['roberta.pooler.dense.weight', 'roberta.pooler.dense.bias']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Asking to truncate to max_length but no maximum length is provided and the model has no predefined maximum length. Default to no truncation.


(1991, 768)

In [ ]:
cols = ['Irony'+str(i) for i in range(768)]
irony_train = pd.DataFrame(irony_feats_train, columns = cols)
irony_train

,Irony0,Irony1,Irony2,Irony3,Irony4,Irony5,Irony6,Irony7,Irony8,Irony9,Irony10,Irony11,Irony12,Irony13,Irony14,Irony15,Irony16,Irony17,Irony18,Irony19,Irony20,Irony21,Irony22,Irony23,Irony24,Irony25,Irony26,Irony27,Irony28,Irony29,Irony30,Irony31,Irony32,Irony33,Irony34,Irony35,Irony36,Irony37,Irony38,Irony39,...,Irony728,Irony729,Irony730,Irony731,Irony732,Irony733,Irony734,Irony735,Irony736,Irony737,Irony738,Irony739,Irony740,Irony741,Irony742,Irony743,Irony744,Irony745,Irony746,Irony747,Irony748,Irony749,Irony750,Irony751,Irony752,Irony753,Irony754,Irony755,Irony756,Irony757,Irony758,Irony759,Irony760,Irony761,Irony762,Irony763,Irony764,Irony765,Irony766,Irony767
0,-0.471292,-0.353281,0.000520,-0.591000,0.697746,0.093900,-0.264468,-0.273179,0.450529,0.188894,-0.786191,-0.617743,-0.311449,-0.866459,0.131818,-1.099350,-1.030521,0.299698,0.485201,-0.613509,-0.124516,0.751994,0.150787,0.319153,0.218940,0.588975,-0.348212,-0.105047,-0.018962,-0.009618,-0.127635,0.149027,0.456927,0.026173,-0.172873,0.523544,0.238605,-0.054298,0.839736,-0.030998,...,-0.584186,0.212610,0.156749,-0.941768,-1.038139,0.177558,0.181305,0.557110,0.061790,-0.353159,0.899544,0.301264,1.101942,-0.171964,-0.046281,0.269243,0.333966,0.177753,0.171535,-0.341656,0.154875,-1.607217,0.911851,-0.996816,0.757478,-0.210797,-0.293119,-0.027593,-0.385141,0.184002,0.253983,-0.935882,-0.152106,-0.308647,0.404672,0.683803,-0.245148,1.000567,0.021097,0.030230
1,-0.173776,-0.641293,-0.235482,0.082421,0.067698,-0.432735,-0.318033,-0.378574,0.457209,-0.249211,-1.653063,-0.807138,0.253787,-0.582611,-0.231862,-0.982045,-0.439636,0.113082,0.959515,-0.675918,-0.072954,0.236668,-0.111898,0.239510,-0.689716,0.003946,0.056775,-0.050179,-0.046874,0.213368,-0.536352,0.220668,0.720704,0.340931,-0.229836,0.219402,0.188598,-0.190553,-0.710029,0.078320,...,-0.008119,0.126614,-0.164805,0.657383,-0.582084,1.096977,0.033940,0.572728,0.272078,-0.327178,1.312637,-0.072290,0.253529,-0.338432,0.116460,0.061419,-0.141006,0.353893,0.284806,-0.156673,-0.807721,-1.486033,0.109425,-0.181164,0.327037,0.060229,0.861837,0.017246,-0.711253,0.288718,1.110680,-0.844379,0.183071,-0.544601,0.176686,0.574355,-0.708552,-0.258226,-0.154422,-0.063448
2,-0.632326,-0.712767,-0.332371,0.002248,0.031701,0.070027,-0.200741,-0.310232,0.488640,0.056382,-1.535152,0.420681,0.171316,-0.384256,-0.313007,-0.970387,-1.101835,0.200696,1.074786,-0.952831,-0.024575,0.575122,-0.184765,0.375134,-0.791742,-0.004247,-0.016209,-0.109050,-0.081582,0.474396,-0.388668,0.297056,0.797782,-0.006649,-0.255695,0.112590,0.300483,-0.033071,-0.544848,0.340234,...,-0.446231,0.151635,0.191382,0.672469,-0.636437,0.200567,0.391596,0.179175,0.593622,-0.346587,1.320112,-0.088991,0.861061,-0.612992,0.331405,-0.028476,0.011264,0.191598,0.230901,-0.577305,0.116680,-1.906403,0.462564,-0.073243,1.228099,-0.196941,0.529654,0.114065,-0.491322,0.056932,0.930114,-0.600814,-0.405208,-0.259786,0.416995,0.610659,-0.541950,-0.056693,-0.208665,-0.326956
3,-0.237255,-0.425166,-0.059811,-0.282922,0.784967,-0.214543,-0.360208,-0.094840,0.359855,-0.103600,-1.284849,-0.304752,-0.008757,-0.169599,-0.165832,-0.689721,0.037220,0.141062,0.764920,-1.023553,0.276134,0.303001,-0.322640,-0.064626,-0.531214,0.150509,-0.103727,0.102857,-0.154476,0.769529,-0.338746,0.261895,0.513800,-0.161160,-0.091394,0.018369,0.197189,-0.231763,-0.401342,0.122670,...,0.187848,-0.311138,-0.094613,0.312931,-0.809709,0.673215,0.203691,0.881549,0.117129,-0.334042,0.974980,-0.045489,0.202240,-0.247238,-0.035140,-0.170411,-0.098894,0.149153,0.428695,-0.466506,-0.049711,-0.544786,-0.055236,0.189383,0.539685,0.358871,0.297094,-0.097512,-0.501298,0.142640,0.216332,-0.388565,0.066418,-0.232666,0.061033,0.567080,-0.180974,0.157965,-0.220183,-0.198878
4,-0.041582,-0.282642,0.033622,-0.665924,0.473944,-0.004473,-0.379165,-0.393312,0.087393,-0.111564,-0.366273,-0.474814,0.000623,-0.654544,-0.353335,-0.789095,-0.161047,0.195358,-0.011365,-0.392667,-0.315843,0.443165,-0.444626,0.246089,-0.085014,-0.058546,-

In [ ]:
irony_train.to_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Irony.csv', index=False)


#Biden Stance Features(768 /sample)


In [ ]:
import nltk
import numpy as np
biden_feats_train = np.array(biden_feats_train)
from sentence_transformers import SentenceTransformer
st = SentenceTransformer('kornosk/bert-election2020-twitter-stance-biden')
biden_feats_train = []
for index, row in df_train.iterrows():
  biden_feats_train.append(st.encode(row['text']))
biden_feats_train = np.array(biden_feats_train)
biden_feats_train.shape

In [ ]:
cols = ['Biden'+str(i) for i in range(768)]
biden_train = pd.DataFrame(biden_feats_train, columns = cols)
biden_train

,Biden0,Biden1,Biden2,Biden3,Biden4,Biden5,Biden6,Biden7,Biden8,Biden9,Biden10,Biden11,Biden12,Biden13,Biden14,Biden15,Biden16,Biden17,Biden18,Biden19,Biden20,Biden21,Biden22,Biden23,Biden24,Biden25,Biden26,Biden27,Biden28,Biden29,Biden30,Biden31,Biden32,Biden33,Biden34,Biden35,Biden36,Biden37,Biden38,Biden39,...,Biden728,Biden729,Biden730,Biden731,Biden732,Biden733,Biden734,Biden735,Biden736,Biden737,Biden738,Biden739,Biden740,Biden741,Biden742,Biden743,Biden744,Biden745,Biden746,Biden747,Biden748,Biden749,Biden750,Biden751,Biden752,Biden753,Biden754,Biden755,Biden756,Biden757,Biden758,Biden759,Biden760,Biden761,Biden762,Biden763,Biden764,Biden765,Biden766,Biden767
0,-0.192171,-0.175991,0.571196,-0.489674,0.467962,0.093435,0.281182,0.689310,0.621131,-0.427144,0.030005,-0.412188,-0.467711,0.116423,-0.725940,0.252235,-0.260069,-0.249069,0.084257,0.068713,-0.601276,0.145681,-0.089388,0.667946,0.758650,0.433282,-0.029174,0.687011,-0.072728,-0.381053,0.227724,0.391782,-0.229141,-0.227810,0.531922,0.466828,0.414276,-0.051787,0.002286,0.251999,...,-0.534803,0.019880,-0.713017,0.154906,-0.315743,0.009973,-0.001753,0.588609,-0.268369,-0.112263,0.321960,-0.081433,0.143531,0.134090,0.068102,-1.069961,0.072352,-0.095728,-0.702690,-0.750595,0.402585,0.223596,-0.319638,0.158280,0.863017,0.118146,0.173723,-0.179035,-0.015388,0.260320,-0.214892,0.264299,0.586318,-0.044728,-0.260310,0.114265,-0.099405,-0.395467,-0.540203,-0.185408
1,0.252564,0.196137,0.588770,-0.376631,0.585159,0.087714,-0.520417,0.774705,0.137179,0.138655,0.083985,-0.123292,0.031911,-0.034609,-0.886867,0.119494,-0.378223,-0.386168,0.006549,-0.256174,0.549153,0.317650,0.097510,0.404536,0.301395,0.311398,0.252025,0.192625,-0.220705,-0.132477,0.093538,0.164701,-0.167619,0.190380,0.243428,0.227160,0.486082,-0.138807,-0.312378,-0.240740,...,-0.392971,-0.455465,-0.421737,-0.011858,-0.133968,-0.106870,-0.114131,-0.385612,-0.400236,0.128552,-0.016964,0.026405,-0.283658,-0.034536,0.036052,-0.707886,-0.068418,-0.125227,-0.729529,-0.670569,0.597918,-0.411114,-0.585722,0.032555,0.820593,-0.136645,0.523351,-0.114138,-0.122703,-0.004156,-0.393847,0.459100,-0.051851,0.190945,-0.338563,-0.229865,-0.150425,-0.458121,-0.176310,-0.055115
2,0.081717,0.308373,-0.014140,-0.231475,0.150347,-0.051002,0.011094,0.350689,0.013931,-0.090363,0.043836,-0.325201,-0.258660,0.142756,-0.772271,0.099260,-0.137961,-0.248939,-0.058764,0.170765,-0.236715,0.128311,-0.085164,0.463866,0.303314,0.145897,-0.057789,0.366294,0.093469,-0.088724,0.253605,0.361259,-0.443301,-0.264383,0.280043,0.195352,0.159348,0.025071,-0.273018,0.174230,...,-0.477702,-0.175030,-0.241542,-0.384360,-0.197197,-0.201184,0.059660,-0.398764,-0.058556,-0.188428,0.008584,0.222813,-0.567219,-0.037905,0.171912,-0.863697,-0.356112,0.071909,-0.619254,-0.627521,0.328410,0.241314,-0.173462,0.081799,0.728936,0.245532,0.234071,-0.251509,-0.004059,-0.583031,-0.174294,0.079121,0.040618,0.367331,-0.082432,0.006193,-0.129231,-0.495398,-0.530584,0.043148
3,0.355196,0.322968,-0.096971,0.018843,0.140393,0.273076,-0.315469,0.267737,-0.273957,0.264952,-0.029794,-0.058499,-0.332016,0.322693,-0.802344,-0.085871,0.236483,-0.044494,0.114217,0.123859,-0.096582,0.320372,-0.344987,0.426812,0.216454,0.043683,0.436618,-0.395470,-0.365061,-0.200043,0.103881,-0.287604,-0.506508,-0.044172,-0.011813,0.609596,0.426956,-0.425722,-0.335957,-0.215075,...,-0.102603,-0.411501,-0.311751,0.011043,-0.444560,-0.350008,-0.089629,-0.213395,-0.718221,0.104648,-0.383359,-0.059258,-0.221384,-0.208912,-0.065230,-0.636063,0.207478,-0.129575,-0.513458,-0.544514,0.743678,-0.218601,-0.473417,0.340127,0.584338,0.261425,0.258828,-0.383195,-0.034908,0.239972,-0.104522,0.255846,-0.070545,-0.385973,-0.412432,-0.146713,0.027013,-0.239316,0.321543,-0.037489
4,0.314612,0.361368,-0.200459,-0.513509,0.535894,0.567573,0.379866,0.446590,-0.080977,0.017504,0.189904,-0.112816,-0.434057,0.352123,-0.832184,0.005212,0.223607,-0.371676,0.248699,-0.031537,-0.182015,0.227944,-0.282624,0.803208,0.480061,0.095574,-0

In [ ]:
biden_train.to_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Biden.csv', index=False)


#Sarcasm(1 /sample)

In [ ]:
from transformers import AutoTokenizer, T5ForConditionalGeneration
from transformers import AutoTokenizer, AutoModelWithLMHead
tokenizer = AutoTokenizer.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")
model = AutoModelWithLMHead.from_pretrained("mrm8488/t5-base-finetuned-sarcasm-twitter")

/usr/local/lib/python3.7/dist-packages/transformers/models/auto/modeling_auto.py:762: FutureWarning: The class `AutoModelWithLMHead` is deprecated and will be removed in a future version. Please use `AutoModelForCausalLM` for causal language models, `AutoModelForMaskedLM` for masked language models and `AutoModelForSeq2SeqLM` for encoder-decoder models.
  FutureWarning,


In [ ]:
def eval(text):
  input_ids = tokenizer.encode(text + '</s>', return_tensors='pt')
  output = model.generate(input_ids=input_ids, max_length=3)
  dec = [tokenizer.decode(ids) for ids in output]
  label = dec[0]
  #print(label)
  if 'normal' in label:
    return 0
  else:
    return 1
df_train['sarcasm'] = df_train['text'].apply(eval)
df_train.drop(columns = df_train.columns[:-1], inplace=True)
df_train

,sarcasm
0,0
1,0
2,0
3,0
4,1
...,...
1986,1
1987,0
1988,1
1989,0


In [ ]:
df_train.to_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Sarcasm.csv', index=False)

#Before moving to MMN, trying previous sklearn and NN models

In [ ]:
X_biden = pd.read_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Biden.csv').values
X_humor = pd.read_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Humor.csv').values
X_irony = pd.read_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Irony.csv').values
X_sarcasm = pd.read_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Sarcasm.csv').values
X_sentiment = pd.read_csv('/content/drive/MyDrive/meme_classification_data/MMN_Features/train_Sentiment.csv').values
y = pd.read_csv('/content/drive/MyDrive/meme_classification_data/train.csv')['label_num']
X_biden.shape, X_humor.shape, X_irony.shape, X_sarcasm.shape, X_sentiment.shape, y.shape


((1991, 768), (1991, 256), (1991, 768), (1991, 1), (1991, 768), (1991,))

In [ ]:
import numpy as np
from sklearn.decomposition import PCA
X = np.concatenate((X_biden, X_irony, X_sarcasm, X_sentiment), axis = 1)
pca = PCA(n_components=400)
pca.fit(X)
X = pca.transform(X)
X.shape
X = np.concatenate((X, X_humor), axis=1)
X.shape

(1991, 656)

In [ ]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from xgboost import XGBClassifier
from sklearn.svm import SVC
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)
models=[SVC(kernel='rbf'), SVC(kernel = 'linear'), SVC(kernel = 'poly', degree = 8),RandomForestClassifier(),DecisionTreeClassifier(), XGBClassifier()]
names = ['SVM-rbf','SVM-linear','SVM-poly','Random-Forest','Decision-Tree', 'XGB']
for i in range(len(models)):
  models[i].fit(X_train,y_train)
  print("MODEL: ", names[i])
  print("On Train")
  print(classification_report(y_train,models[i].predict(X_train)))
  print("On Test")
  print(classification_report(y_test, models[i].predict(X_test)))

MODEL:  SVM-rbf
On Train
              precision    recall  f1-score   support

           0       0.80      0.59      0.68       431
           1       0.64      0.79      0.71       481
           2       0.68      0.68      0.68       481

    accuracy                           0.69      1393
   macro avg       0.71      0.69      0.69      1393
weighted avg       0.70      0.69      0.69      1393

On Test
              precision    recall  f1-score   support

           0       0.39      0.25      0.30       173
           1       0.40      0.55      0.46       208
           2       0.44      0.41      0.42       217

    accuracy                           0.41       598
   macro avg       0.41      0.40      0.40       598
weighted avg       0.41      0.41      0.40       598

MODEL:  SVM-linear
On Train
              precision    recall  f1-score   support

           0       0.80      0.85      0.82       431
           1       0.84      0.80      0.82       481
           2  

In [ ]:
from sklearn.model_selection import train_test_split
from keras.utils.np_utils import to_categorical   
categorical_labels = to_categorical(y, num_classes=3)
X_train, X_test, y_train, y_test = train_test_split(X, categorical_labels, test_size=0.2, random_state=42)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.125, random_state=42)
X_train.shape, X_val.shape, X_test.shape, y_train.shape, y_val.shape, y_test.shape

((1393, 656), (199, 656), (399, 656), (1393, 3), (199, 3), (399, 3))

In [ ]:
from keras.models import Sequential
from keras.layers import Dense, Dropout
from keras.optimizers import RMSprop,Adam
model=Sequential() 
model.add(Dense(30,activation='relu',input_shape=(656,),kernel_initializer='he_normal'))
model.add(Dropout(0.3))
#model.add(Dense(20,activation='sigmoid'))
model.add(Dense(8,activation='sigmoid'))
model.add(Dense(3,activation='softmax'))
model.summary()

Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_27 (Dense)             (None, 30)                19710     
_________________________________________________________________
dropout_7 (Dropout)          (None, 30)                0         
_________________________________________________________________
dense_28 (Dense)             (None, 8)                 248       
_________________________________________________________________
dense_29 (Dense)             (None, 3)                 27        
Total params: 19,985
Trainable params: 19,985
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.compile(loss='categorical_crossentropy',
              optimizer=Adam(),
              metrics=['accuracy','AUC'])


history = model.fit(X_train, y_train,
                    batch_size=128,
                    epochs=30,
                    verbose=1,
                    validation_data=(X_val, y_val))

Epoch 1/30
11/11 [==============================] - 1s 31ms/step - loss: 1.1982 - accuracy: 0.3271 - auc: 0.4956 - val_loss: 1.0966 - val_accuracy: 0.3869 - val_auc: 0.5481
Epoch 2/30
11/11 [==============================] - 0s 7ms/step - loss: 1.1139 - accuracy: 0.3577 - auc: 0.5257 - val_loss: 1.1000 - val_accuracy: 0.3668 - val_auc: 0.5360
Epoch 3/30
11/11 [==============================] - 0s 7ms/step - loss: 1.0984 - accuracy: 0.3682 - auc: 0.5445 - val_loss: 1.1014 - val_accuracy: 0.3518 - val_auc: 0.5336
Epoch 4/30
11/11 [==============================] - 0s 8ms/step - loss: 1.0879 - accuracy: 0.3845 - auc: 0.5659 - val_loss: 1.0957 - val_accuracy: 0.3618 - val_auc: 0.5483
Epoch 5/30
11/11 [==============================] - 0s 8ms/step - loss: 1.0808 - accuracy: 0.4173 - auc: 0.5847 - val_loss: 1.0944 - val_accuracy: 0.3317 - val_auc: 0.5555
Epoch 6/30
11/11 [==============================] - 0s 8ms/step - loss: 1.0666 - accuracy: 0.4470 - auc: 0.6167 - val_loss: 1.0948 - val_ac

In [ ]:
from sklearn.metrics import classification_report
y_pred = model.predict(X_test, batch_size=64, verbose=1)
y_pred = np.argmax(y_pred, axis = 1)
y_true = np.argmax(y_test, axis = 1)
print(classification_report(y_true,y_pred))

7/7 [==============================] - 0s 2ms/step
              precision    recall  f1-score   support

           0       0.40      0.36      0.38       117
           1       0.39      0.43      0.41       136
           2       0.45      0.45      0.45       146

    accuracy                           0.41       399
   macro avg       0.41      0.41      0.41       399
weighted avg       0.41      0.41      0.41       399

